In [1]:
#pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [25]:
import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import time
import datetime  
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn
from transformers import *
from kobert_tokenizer import KoBERTTokenizer
# for KoBART
from kobart import get_kobart_tokenizer 
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer 
import torch.nn.functional as nnf

In [2]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") 

train.shape, test.shape, train['label'].nunique()

((174304, 13), (43576, 12), 46)

In [3]:
submission = pd.read_csv("sample_submission.csv")

In [4]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

In [5]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total

In [6]:
train['요약문_내용'] = train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과']

In [7]:
train['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)

In [8]:
train['사업명'].fillna('NAN',inplace=True) 
train['사업_부처명'].fillna('NAN',inplace=True) 
train['내역사업명'].fillna('NAN',inplace=True) 
train['과제명'].fillna('NAN',inplace=True) 
train['요약문_한글키워드'].fillna('NAN',inplace=True)

In [9]:
contents = train['요약문_내용'].values 
feature1 = train['사업명'].values 
feature2 = train['사업_부처명'].values 
feature3 = train['내역사업명'].values 
feature4 = train['과제명'].values 
feature5 = train['요약문_한글키워드'].values 
feature6 = train['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['label'].append(feature6[i])

100%|██████████| 174304/174304 [02:36<00:00, 1113.65it/s]


In [10]:
train_data = pd.DataFrame(train_data)
train_data.head(5)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24
2,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통한 ...,0
3,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",저항성 극복 기전을 규명 환자조직 동물실험 세포실험을 통해 대장암에 특이적으로 조절...,0
4,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",내성 유발 유전자를 발굴하고 저항성 극복 기전을 규명 추후 기반 항암화학요법 치료효...,0


In [11]:
kobart_tokenizer = get_kobart_tokenizer()

def bart_tokenizer(sent, MAX_LEN):  
    tokens = [kobart_tokenizer.bos_token] + kobart_tokenizer.tokenize(sent) + [kobart_tokenizer.eos_token] 
    encoder_input_ids = kobart_tokenizer.convert_tokens_to_ids(tokens) 
    attention_mask = [1] * len(encoder_input_ids) 
    
    if len(encoder_input_ids) >= MAX_LEN:  
        encoder_input_ids = encoder_input_ids[:129] + encoder_input_ids[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]
        print("Long Text!! Using Head+Tail Truncation") 
    elif len(encoder_input_ids) < MAX_LEN: 
        while len(encoder_input_ids) < MAX_LEN:
            encoder_input_ids += [kobart_tokenizer.pad_token_id]
            attention_mask += [0]
    
    return encoder_input_ids, attention_mask

using cached model


In [12]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['요약문'] 

train_data.head(2)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label,data
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...


In [13]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values

In [14]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

In [15]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask = bart_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass


 11%|█         | 173124/1638867 [05:37<38:41, 631.35it/s]  

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 14%|█▍        | 233813/1638867 [07:28<59:33, 393.21it/s]  

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 15%|█▌        | 251077/1638867 [08:05<52:59, 436.44it/s]  

...

 83%|████████▎ | 1353102/1638867 [46:23<11:29, 414.57it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 84%|████████▍ | 1373797/1638867 [47:12<10:09, 434.98it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 1638867/1638867 [57:22<00:00, 476.02it/s]


In [16]:
input_ids = torch.tensor(input_ids, dtype=int)
attention_masks = torch.tensor(attention_masks, dtype=int)
labels = torch.tensor(labels, dtype=int)

In [17]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 12345, test_size = VALID_SPLIT, stratify=labels) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 12345, test_size = VALID_SPLIT, stratify=labels) 



train_inputs.shape, train_attention_mask.shape, train_labels.shape, val_inputs.shape, val_attention_mask.shape, val_labels.shape


(torch.Size([1474980, 512]),
 torch.Size([1474980, 512]),
 torch.Size([1474980]),
 torch.Size([163887, 512]),
 torch.Size([163887, 512]),
 torch.Size([163887]))

In [18]:
batch_size = 16
train_data = TensorDataset(train_inputs, train_attention_mask, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [19]:
model = BartForSequenceClassification.from_pretrained(get_pytorch_kobart_model(), num_labels=46)
model.cuda()

using cached model


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at /root/kobart/kobart_from_pretrained and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  

In [20]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 10

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


device = torch.device("cuda")


# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch 
        
        # Forward 수행                
        outputs = model(input_ids=b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(input_ids=b_input_ids, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)            
    print("  Average validation loss: {}".format(avg_val_loss))
    print("  Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "BART_stratified_" + str(epoch_i + 1)) 
    

print("")
print("Training complete!") 



======== Epoch 1 / 10 ========
Training...
  Batch   100  of  92,187.    Elapsed: 0:01:15.
  current average loss = 1.1185031375288963
  Batch   200  of  92,187.    Elapsed: 0:02:30.
  current average loss = 1.100680648162961
  Batch   300  of  92,187.    Elapsed: 0:03:43.
  current average loss = 1.0552449949334066
  Batch   400  of  92,187.    Elapsed: 0:04:57.
  current average loss = 1.018781632259488
  Batch   500  of  92,187.    Elapsed: 0:06:09.
  current average loss = 0.9610074435770511
  Batch   600  of  92,187.    Elapsed: 0:07:23.
  current average loss = 0.9171966115074853
  Batch   700  of  92,187.    Elapsed: 0:08:36.
  current average loss = 0.8725727490854582
  Batch   800  of  92,187.    Elapsed: 0:09:49.
  current average loss = 0.8339885135402437
  Batch   900  of  92,187.    Elapsed: 0:11:01.
  current average loss = 0.803570823026821
  Batch 1,000  of  92,187.    Elapsed: 0:12:13.
  current average loss = 0.7720621097227558
  Batch 1,100  of  92,187.    Elapsed: 

  Batch 9,000  of  92,187.    Elapsed: 1:48:36.
  current average loss = 0.4113157977239591
  Batch 9,100  of  92,187.    Elapsed: 1:49:48.
  current average loss = 0.41011079376479
  Batch 9,200  of  92,187.    Elapsed: 1:51:01.
  current average loss = 0.4087587103922598
  Batch 9,300  of  92,187.    Elapsed: 1:52:13.
  current average loss = 0.40775172018072997
  Batch 9,400  of  92,187.    Elapsed: 1:53:25.
  current average loss = 0.4062983935030346
  Batch 9,500  of  92,187.    Elapsed: 1:54:36.
  current average loss = 0.40527094564515803
  Batch 9,600  of  92,187.    Elapsed: 1:55:48.
  current average loss = 0.4040248327478184
  Batch 9,700  of  92,187.    Elapsed: 1:57:00.
  current average loss = 0.4029845603715782
  Batch 9,800  of  92,187.    Elapsed: 1:58:13.
  current average loss = 0.4022117824612033
  Batch 9,900  of  92,187.    Elapsed: 1:59:25.
  current average loss = 0.4008155823001318
  Batch 10,000  of  92,187.    Elapsed: 2:00:38.
  current average loss = 0.3999

  Batch 17,800  of  92,187.    Elapsed: 3:34:29.
  current average loss = 0.33663671077164664
  Batch 17,900  of  92,187.    Elapsed: 3:35:40.
  current average loss = 0.3360297726645419
  Batch 18,000  of  92,187.    Elapsed: 3:36:52.
  current average loss = 0.335440526684329
  Batch 18,100  of  92,187.    Elapsed: 3:38:04.
  current average loss = 0.33479819387330956
  Batch 18,200  of  92,187.    Elapsed: 3:39:17.
  current average loss = 0.33424194984888633
  Batch 18,300  of  92,187.    Elapsed: 3:40:29.
  current average loss = 0.33365920766331497
  Batch 18,400  of  92,187.    Elapsed: 3:41:40.
  current average loss = 0.3329701061576887
  Batch 18,500  of  92,187.    Elapsed: 3:42:53.
  current average loss = 0.3324158692508527
  Batch 18,600  of  92,187.    Elapsed: 3:44:05.
  current average loss = 0.3317290961441818
  Batch 18,700  of  92,187.    Elapsed: 3:45:17.
  current average loss = 0.33122401330390644
  Batch 18,800  of  92,187.    Elapsed: 3:46:29.
  current average

  Batch 26,600  of  92,187.    Elapsed: 5:20:04.
  current average loss = 0.29342449942419674
  Batch 26,700  of  92,187.    Elapsed: 5:21:16.
  current average loss = 0.2930199006696869
  Batch 26,800  of  92,187.    Elapsed: 5:22:29.
  current average loss = 0.29250922511242905
  Batch 26,900  of  92,187.    Elapsed: 5:23:43.
  current average loss = 0.2921453199772918
  Batch 27,000  of  92,187.    Elapsed: 5:24:55.
  current average loss = 0.29172116331100306
  Batch 27,100  of  92,187.    Elapsed: 5:26:07.
  current average loss = 0.2912543705216913
  Batch 27,200  of  92,187.    Elapsed: 5:27:19.
  current average loss = 0.2909044081548114
  Batch 27,300  of  92,187.    Elapsed: 5:28:31.
  current average loss = 0.2903867163206792
  Batch 27,400  of  92,187.    Elapsed: 5:29:43.
  current average loss = 0.2898825308864462
  Batch 27,500  of  92,187.    Elapsed: 5:30:55.
  current average loss = 0.28948416597655635
  Batch 27,600  of  92,187.    Elapsed: 5:32:07.
  current average

  Batch 35,400  of  92,187.    Elapsed: 7:05:39.
  current average loss = 0.2617981507567816
  Batch 35,500  of  92,187.    Elapsed: 7:06:52.
  current average loss = 0.26148141332353303
  Batch 35,600  of  92,187.    Elapsed: 7:08:05.
  current average loss = 0.2612240803433774
  Batch 35,700  of  92,187.    Elapsed: 7:09:17.
  current average loss = 0.26098412728454207
  Batch 35,800  of  92,187.    Elapsed: 7:10:30.
  current average loss = 0.26071326699996444
  Batch 35,900  of  92,187.    Elapsed: 7:11:42.
  current average loss = 0.2604809865389881
  Batch 36,000  of  92,187.    Elapsed: 7:12:53.
  current average loss = 0.2601449726553045
  Batch 36,100  of  92,187.    Elapsed: 7:14:04.
  current average loss = 0.2597861243871691
  Batch 36,200  of  92,187.    Elapsed: 7:15:17.
  current average loss = 0.2595744223129911
  Batch 36,300  of  92,187.    Elapsed: 7:16:29.
  current average loss = 0.2592741721705004
  Batch 36,400  of  92,187.    Elapsed: 7:17:40.
  current average 

...

  Batch 86,300  of  92,187.    Elapsed: 15:53:48.
  current average loss = 0.011186060780773627
  Batch 86,400  of  92,187.    Elapsed: 15:54:55.
  current average loss = 0.011189438119726216
  Batch 86,500  of  92,187.    Elapsed: 15:56:01.
  current average loss = 0.011179983474985774
  Batch 86,600  of  92,187.    Elapsed: 15:57:08.
  current average loss = 0.011185830949293154
  Batch 86,700  of  92,187.    Elapsed: 15:58:14.
  current average loss = 0.011180044253571511
  Batch 86,800  of  92,187.    Elapsed: 15:59:20.
  current average loss = 0.01117645038157948
  Batch 86,900  of  92,187.    Elapsed: 16:00:27.
  current average loss = 0.011172345799601894
  Batch 87,000  of  92,187.    Elapsed: 16:01:33.
  current average loss = 0.011173535304778088
  Batch 87,100  of  92,187.    Elapsed: 16:02:40.
  current average loss = 0.01119210000913699
  Batch 87,200  of  92,187.    Elapsed: 16:03:46.
  current average loss = 0.011186486604622346
  Batch 87,300  of  92,187.    Elapsed: 16

  Batch 2,600  of  92,187.    Elapsed: 0:28:44.
  current average loss = 0.008786886096625564
  Batch 2,700  of  92,187.    Elapsed: 0:29:51.
  current average loss = 0.008915235126979934
  Batch 2,800  of  92,187.    Elapsed: 0:30:57.
  current average loss = 0.00874258247157757
  Batch 2,900  of  92,187.    Elapsed: 0:32:03.
  current average loss = 0.008903146940440209
  Batch 3,000  of  92,187.    Elapsed: 0:33:09.
  current average loss = 0.008817293515045397
  Batch 3,100  of  92,187.    Elapsed: 0:34:15.
  current average loss = 0.009036897749043683
  Batch 3,200  of  92,187.    Elapsed: 0:35:22.
  current average loss = 0.009072873412921113
  Batch 3,300  of  92,187.    Elapsed: 0:36:28.
  current average loss = 0.008976512455402081
  Batch 3,400  of  92,187.    Elapsed: 0:37:34.
  current average loss = 0.008926736469971422
  Batch 3,500  of  92,187.    Elapsed: 0:38:40.
  current average loss = 0.008855735097534501
  Batch 3,600  of  92,187.    Elapsed: 0:39:47.
  current ave

  Batch 11,400  of  92,187.    Elapsed: 2:06:23.
  current average loss = 0.008636623857854023
  Batch 11,500  of  92,187.    Elapsed: 2:07:29.
  current average loss = 0.00873031227823104
  Batch 11,600  of  92,187.    Elapsed: 2:08:35.
  current average loss = 0.008697652814850212
  Batch 11,700  of  92,187.    Elapsed: 2:09:42.
  current average loss = 0.0086605886477559
  Batch 11,800  of  92,187.    Elapsed: 2:10:48.
  current average loss = 0.008616773055666573
  Batch 11,900  of  92,187.    Elapsed: 2:11:54.
  current average loss = 0.008677293888191294
  Batch 12,000  of  92,187.    Elapsed: 2:13:01.
  current average loss = 0.008645055655896329
  Batch 12,100  of  92,187.    Elapsed: 2:14:09.
  current average loss = 0.008637703414998131
  Batch 12,200  of  92,187.    Elapsed: 2:15:15.
  current average loss = 0.00861559665031096
  Batch 12,300  of  92,187.    Elapsed: 2:16:21.
  current average loss = 0.008599636540897765
  Batch 12,400  of  92,187.    Elapsed: 2:17:28.
  cur

  Batch 20,100  of  92,187.    Elapsed: 3:42:49.
  current average loss = 0.008568407278344736


KeyboardInterrupt: 

# Prediction Code

In [21]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values

In [22]:
# Load KoBART
bart_checkpoint = torch.load("BART_stratified_6")
test_bart = BartForSequenceClassification.from_pretrained(get_pytorch_kobart_model(), num_labels=46)
test_bart.load_state_dict(bart_checkpoint)
test_bart.cuda()
print()

using cached model


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at /root/kobart/kobart_from_pretrained and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
bart_probsum = [] 
bart_chunkparts = [] 

# change to eval mode 
test_bart.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    #f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_bart = [] # store probabilities for electra 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        # tokenize test text 
        input_id, attention_mask = bart_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        with torch.no_grad(): 
            outputs = test_bart(input_id, 
                                attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_bart.append(prob)  

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_bart)):  
        for j in range(46): 
            prob_sum[j] += probabilities_bart[i][j]   
    
    bart_probsum.append(prob_sum) 
    bart_chunkparts.append(len(probabilities_bart))


bart_probsum = np.asarray(bart_probsum) 
bart_chunkparts = np.asarray(bart_chunkparts)

print("saving predicted probabilities values") 
np.save('bart_probsum.npy', bart_probsum) 
np.save('bart_chunkparts.npy', bart_chunkparts)


  7%|▋         | 3141/43576 [15:12<3:28:35,  3.23it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [43:02<3:00:29,  3.23it/s]

Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [2:16:18<1:01:26,  4.45it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [3:39:50<00:00,  3.51it/s]  


saving predicted probabilities values
